In [ ]:
# import os
import matplotlib.pyplot as plt
import os
import pandas as pd
from amftrack.util.sys import get_analysis_folders,get_time_plate_info_from_analysis,get_time_hypha_info_from_analysis,get_global_hypha_info_from_analysis 
import numpy as np
import imageio
import os
import cv2
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from amftrack.pipeline.final_analysis.density_wave import get_wave_fit, S, dS, wave, dwave
import matplotlib.patches as mpatches
from random import choice
import matplotlib as mpl
from amftrack.pipeline.final_analysis.density_wave import plot_single_plate,plot_single_plate_biovolume
from amftrack.pipeline.final_analysis.hypha_speed_analysis import *
from amftrack.util.plot import gridplot, make_stat
from helper import *
from load_dataframes import load_all
from plates import plates,timestep
%store -r path_figure
%matplotlib widget
%load_ext autoreload
%autoreload 2
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy
import statsmodels.api as sm
import hashlib
from matplotlib.patches import Ellipse
import logging
plt.style.use('presentation.mplstyle')
import seaborn as sns
import matplotlib.pyplot as plt
logging.getLogger().setLevel(logging.WARNING)




In [ ]:
path_figure = r"C:\Users\coren\Documents\PhD\paper\PandC\Fig2"


In [ ]:
df,final_data = load_all(plates)

In [ ]:
def calculate_derivative(df, column, new_column, window=1):
    # Ensure the DataFrame is sorted by 'unique_id' and 'time_since_begin_h'
    df.sort_values(by=['unique_id', 'time_since_begin_h'], inplace=True)
    
    # Calculate the time difference and value difference over the desired window
    df['time_diff'] = df.groupby('unique_id')['time_since_begin_h'].transform(lambda x: x.diff(window)).fillna(0)
    df['value_diff'] = df.groupby('unique_id')[column].transform(lambda x: x.diff(window)).fillna(0)

    # Calculate the derivative (value difference divided by time difference)
    # Be cautious about dividing by zero. If 'time_diff' is 0, set derivative to NaN or handle accordingly.
    df[new_column] = df.apply(lambda x: x['value_diff'] / x['time_diff'] if x['time_diff'] != 0 else None, axis=1)

    # Drop the helper columns
    df.drop(['time_diff', 'value_diff'], axis=1, inplace=True)
def calculate_integral(df, column, new_column):
    # Calculate the time difference within each group
    df.sort_values(by=['unique_id', 'time_since_begin_h'], inplace=True)
    df['time_since_begin_hour'] = df['time_since_begin_h']
    df['time_diff'] = df.groupby('unique_id')['time_since_begin_hour'].transform(lambda x: x.diff())

    # Calculate the average length density within each group
    df['avg_length_density'] = df.groupby('unique_id')[column].transform(lambda x: x.rolling(window=2).mean())

    # Calculate the "area" (using Trapezoidal rule) for each pair of rows within each group
    df['area'] = df['time_diff'] * df['avg_length_density']

    # Perform the integration (cumulative sum of "area") within each group
    df[new_column] = df.groupby('unique_id')['area'].transform(lambda x: x.cumsum())

    # Drop the helper columns if needed
    df.drop(['time_diff', 'avg_length_density', 'area'], axis=1, inplace=True)
calculate_integral(final_data,f'tot_surface_area_study',f'integrated_SA')
calculate_derivative(final_data,f'tot_biovolume_study',f'derivative_biovolume', window=5)
calculate_derivative(final_data,f'tot_volume',f'derivative_tot_biovolume', window=5)

Vmax = 3e-9*25 #ugP/um2/hour
density = 1e-6 #ug/um3
dry_wet_ratio = 0.3
carbon_ratio = 0.5
carbon_ratio_spore = 0.9
dry_wet_ratio_spore = 0.9
density_spore = 0.9e-6
respiration_ratio = 0.5
final_data['P_rate'] =final_data['tot_surface_area_study']*Vmax
final_data['C_rate'] = final_data['derivative_biovolume']*density*dry_wet_ratio*carbon_ratio/respiration_ratio
final_data['totC_hypha'] = final_data['tot_biovolume_study']*density*dry_wet_ratio*carbon_ratio
final_data['tot_volume'] = final_data['spore_volume']+final_data['tot_biovolume_study']
final_data['totC_spore'] = final_data['spore_volume']*density_spore*dry_wet_ratio_spore*carbon_ratio_spore
final_data['totC_tot'] = final_data['totC_hypha'] + final_data['totC_spore']

final_data = final_data.reset_index(drop=True)
final_data_select = final_data.loc[final_data['time_since_begin_h_shifted']<=150]

max_c_rate_time = final_data_select.loc[final_data_select.groupby('unique_id')['C_rate'].idxmax(), ['unique_id', 'time_since_begin_h_shifted']]

# Step 2: Merge this information back to the original DataFrame
final_data = final_data.merge(max_c_rate_time, on='unique_id', suffixes=('', '_max_c_rate'))

In [ ]:
final_data['time_since_begin_h_shifted_max_c_rate']

In [ ]:
calculate_derivative(final_data,f'tot_surface_area_study',f'derivative_SA', window=5)
calculate_derivative(final_data,f'totC_tot',f'derivative_C', window=5)

calculate_derivative(final_data,f'area_sep_comp',f'derivative_area', window=5)
final_data['instant_SA_dens'] = final_data['derivative_SA']/final_data['derivative_area']
calculate_derivative(final_data,f'tot_biovolume_study',f'derivative_BV', window=5)
final_data['instant_BV_dens'] = final_data['derivative_BV']/final_data['derivative_area']
calculate_derivative(final_data,f'tot_length_study',f'derivative_L', window=5)
final_data['instant_L_dens'] = final_data['derivative_L']/final_data['derivative_area']
final_data['instant_C_dens'] = final_data['derivative_C']/final_data['derivative_area']

final_data['instant_radius'] = final_data['instant_BV_dens']/final_data['instant_SA_dens']*2


In [ ]:
df['FungalSide'] = df['FungalSide'].fillna("100P/100N/100C")
df['FungalSide'] = df['FungalSide'].replace("100P100N100C","100P/100N/100C")
df['FungalSide'] = df['FungalSide'].replace("100P100N100C","100P/100N/100C")

df['treatment'] = df['treatment'].replace("1P100N100C","1P/100N/100C")
df['fungus'] = df['fungus'].replace("A5sp3","A5")
df['category_combination'] = df[['treatment', 'FungalSide',"real_root"]].apply(lambda x: '_'.join(x.astype(str)), axis=1)
df['size_point'] = 20
marker_symbols = {
    "C2": 'circle',
    "A5": 'square',
    "Agg": 'diamond',
    "Clarum": 'cross',
    "C3": 'star',
    
    
    # Add more mappings as needed
}

n_colors = len(df['fungus'].unique())

palette = sns.color_palette("rainbow", n_colors)
strain_to_color = dict(zip(df['fungus'].unique(), palette))
df['full_category_combination'] = df[['fungus','treatment', 'FungalSide',"real_root"]].apply(lambda x: '_'.join(x.astype(str)), axis=1)

# Map the strain to the marker symbol in the DataFrame
df['marker'] = df['fungus'].map(marker_symbols)


In [ ]:
bin_size = 30
cm = 1/2.54  # centimeters in inches
select = final_data
select = final_data.loc[final_data['hull_speed']>0].copy()
select['hull_speed_binned'] = select['hull_speed'].astype(int)//bin_size*bin_size
select= select[select['instant_C_dens']>0]
select= select[select['instant_C_dens']<=0.025]
select= select[select['real_root']=="Carrot Toby"]
# select= select[select['strain']=="C2"]
select = select[select['time_since_begin_h_shifted'] < select['time_since_begin_h_shifted_max_c_rate']+20]
# select = select[select['time_hour_binned'] <= 50]

summary = select.groupby(["strain", 'time_hour_binned']).agg({
    'hull_speed': 'median',
    'instant_SA_dens': 'median',
    'radius': 'median',
    'instant_BV_dens': 'count',
    'instant_L_dens': 'median',
    'instant_C_dens': 'median',
    
}).rename(columns={'instant_BV_dens': 'count'})
summary = summary[summary['count']>7]
summary = summary.reset_index()
fig,ax = plt.subplots(figsize = (6*cm,4*cm))

sns.scatterplot(summary, x='hull_speed', y='instant_C_dens',hue="strain",s=50,palette=strain_to_color,alpha = 0.4)
sns.regplot(data=summary, x='hull_speed', y='instant_C_dens',scatter = False,color = "black")
summary = select.groupby("strain")[['hull_speed','hull_speed_binned','instant_SA_dens','radius','instant_BV_dens','instant_L_dens','instant_C_dens']].median()
sns.scatterplot(summary, x='hull_speed', y='instant_C_dens',hue="strain",s=200,palette=strain_to_color)
ax.legend('')
summary['size_point'] = 20
summary['fungus'] = summary.index
ax.set_xlabel('$v_{wave}$ (um/h)')
ax.set_ylabel('$d_{Carbon}$ \n ($\mu g/mm^2$)')
plt.legend().remove()
# plt.tight_layout()

plt.savefig(os.path.join(path_figure,'Figure2A.pdf'), transparent=True, bbox_inches='tight')


In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming final_data is your DataFrame and it contains a 'unique_id' column
# Group by 'time_hour_binned' and filter out groups with less than 3 unique IDs
tmax = 150
select = final_data[final_data['real_root']=="Carrot Toby"]
select = select[select['time_since_begin_h_shifted'] < select['time_since_begin_h_shifted_max_c_rate']+20]
select = select[select['time_hour_binned'] <= 50]
select = select[select['C_rate'] >= 0]
select = select.copy()
select['time_reshift'] = select['time_hour_binned']+50



grouped = select.groupby('time_hour_binned')
filtered_groups = grouped.filter(lambda x: x['unique_id'].nunique() >= 7)

# Now, use this filtered data for your plots
fig,ax = plt.subplots(figsize = (6*cm,4*cm))
sns.lineplot(data=filtered_groups, x='time_reshift', y='C_rate', hue='strain', ax=ax, palette=strain_to_color, lw=1)

# Mean line plot
sns.lineplot(data=filtered_groups, x='time_reshift', y='C_rate', ax=ax, color='black', estimator='mean', lw=5, ls="-")

ax.set_xlabel('t (hour)')
ax.set_ylabel('$\Phi_C$ \n ($\mu g/h$)')
# ax.set_ylim(0)
plt.legend().remove()
# plt.tight_layout()
# Save the figure
plt.savefig(os.path.join(path_figure, 'Figure2B.pdf'), transparent=True, bbox_inches='tight')


In [ ]:
fig,ax = plt.subplots()
n_colors = len(final_data['strain'].unique())

tmax = 150
select = final_data[final_data['real_root']=="Carrot Toby"]
select = select[select['time_since_begin_h_shifted'] < select['time_since_begin_h_shifted_max_c_rate']]

sns.lineplot(data=select, x='time_hour_binned', y=select['C_rate'], hue='strain',ax=ax, palette=strain_to_color,lw = 1)
sns.lineplot(data=select, x='time_hour_binned', y=select['C_rate'], ax=ax, color = 'black', estimator='mean', lw=5, ls="-")

# ax.legend().remove()
ax.set_xlabel('t (hour)')
ax.set_ylabel('BV ($\mu m^3$)')
ax.set_ylim(0,0.4)
ax.set_ylabel('C provision rate ($\mu g/h$)')
ax.set_xlabel('t (hour)')
# ax.set_xlim(-40,100)
plt.tight_layout()
plt.savefig(os.path.join(path_figure,'Figure2B.pdf'), transparent=True, bbox_inches='tight')


In [ ]:
bin_size = 20

select = final_data
select = final_data.loc[final_data['hull_speed']>0].copy()
select['hull_speed_binned'] = select['hull_speed'].astype(int)//bin_size*bin_size

# select= final_data[final_data['real_root']=="Carrot Toby"]
select= select[select['real_root']=="Carrot Toby"]
# select= select[select['strain']=="C2"]
select = select[select['time_since_begin_h_shifted'] < select['time_since_begin_h_shifted_max_c_rate']]

# select= select[select['time_hour_binned']<=160]
select= select[select['instant_C_dens']<=0.02]
select= select[select['instant_C_dens']>0]
# select= select[select['time_hour_binned']>=20]
summary = select.groupby("strain")[['hull_speed','hull_speed_binned','instant_SA_dens','radius','instant_BV_dens','instant_L_dens','instant_C_dens']].median()

summary['size_point'] = 20
summary['fungus'] = summary.index


fig,ax = plt.subplots(figsize = (6,6))


# sns.scatterplot(select, x='hull_speed', y='instant_C_dens',hue="strain",s=5,legend = False,palette=strain_to_color)
sns.scatterplot(summary, x='hull_speed', y='instant_C_dens',hue="strain",s=300,palette=strain_to_color)
# select = select.loc[select['hull_speed']<320]
# select = select.loc[select['hull_speed']>160]
select= select[select['instant_C_dens']<=0.02]

# sns.regplot(select, x='hull_speed', y='instant_C_dens',scatter = False)

sns.lineplot(select, x='hull_speed_binned', y='instant_C_dens',estimator = "median")
ax.set_xlabel('$v_{wave}$ (um/h)')
ax.set_ylabel(r'biomass density ($\mu g/mm^2$)')

plt.tight_layout()


In [ ]:
bin_size = 30

select = final_data
select = final_data.loc[final_data['hull_speed']>0].copy()
select['hull_speed_binned'] = select['hull_speed'].astype(int)//bin_size*bin_size
select= select[select['instant_C_dens']>0]
select= select[select['instant_C_dens']<=0.06]
select= select[select['time_hour_binned']>=50]

summary = select.groupby(["strain", 'hull_speed_binned']).agg({
    'hull_speed': 'median',
    'instant_SA_dens': 'median',
    'radius': 'median',
    'instant_BV_dens': 'median',
    'instant_L_dens': 'median',
    'instant_C_dens': 'median',
    'hull_speed': 'count'  # This will count the number of rows in each group
}).rename(columns={'hull_speed': 'count'})
summary = summary[summary['count']>30]
summary = summary.reset_index()

fig,ax = plt.subplots(figsize = (6,6))
sns.scatterplot(summary, x='hull_speed_binned', y='instant_C_dens',hue="strain",s=300,palette=strain_to_color)
sns.regplot(data=summary, x='hull_speed_binned', y='instant_C_dens',scatter = False)



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample

bin_size = 20

select = final_data
select = final_data.loc[final_data['hull_speed']>0].copy()
select['hull_speed_binned'] = select['hull_speed'].astype(int)//bin_size*bin_size
counts = select.groupby('strain')['strain'].transform('count')

# Calculate the weight as the reciprocal of the count
select['weight'] = 1 / counts

# select= final_data[final_data['real_root']=="Carrot Toby"]
select= select[select['real_root']=="Carrot Toby"]
# select= select[select['strain']=="C2"]
select = select[select['time_since_begin_h_shifted'] < select['time_since_begin_h_shifted_max_c_rate']]

# select= select[select['time_hour_binned']<=160]
select= select[select['instant_C_dens']<=0.03]
select= select[select['instant_C_dens']>0]
# select= select[select['time_hour_binned']>=50]
summary = select.groupby("strain")[['hull_speed','hull_speed_binned','instant_SA_dens','radius','instant_BV_dens','instant_L_dens','instant_C_dens']].median()

summary['size_point'] = 20
summary['fungus'] = summary.index
# Bootstrapping parameters
n_bootstraps = 10
bootstrapped_medians = {strain: {'hull_speed': [], 'instant_C_dens': []} for strain in select['strain'].unique()}

# Perform Bootstrapping
for _ in range(n_bootstraps):
    for strain, group in select.groupby('strain'):
        sample = group.sample(n=len(group), replace=True)  # Bootstrap sample
        bootstrapped_medians[strain]['hull_speed'].append(sample['hull_speed'].median())
        bootstrapped_medians[strain]['instant_C_dens'].append(sample['instant_C_dens'].median())

# Calculate errors (standard deviation of the bootstrapped medians)
errors = {strain: {'hull_speed': np.std(bootstrapped_medians[strain]['hull_speed']),
                   'instant_C_dens': np.std(bootstrapped_medians[strain]['instant_C_dens'])}
          for strain in bootstrapped_medians}

# Plotting
fig, ax = plt.subplots(figsize=(6, 6))

for strain, color in strain_to_color.items():
    strain_data = summary.loc[summary['fungus'] == strain]
    ax.errorbar(strain_data['hull_speed'], strain_data['instant_C_dens'],
                xerr=errors[strain]['hull_speed'], yerr=errors[strain]['instant_C_dens'],
                fmt='o', color=color, label=strain, markersize=10)


sns.regplot(summary, x='hull_speed', y='instant_C_dens',scatter = False,ci=None)
select= select[select['instant_C_dens']<=0.05]

# sns.scatterplot(select, x='hull_speed', y='instant_C_dens',hue="strain",s=5,legend = False,palette=strain_to_color,alpha = 0.2)
# sns.regplot(select, x='hull_speed', y='instant_C_dens',scatter = False)

ax.set_xlabel('$v_{wave}$ (um/h)')
ax.set_ylabel(r'biomass density ($\mu g/mm^2$)')
ax.set_ylim
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots()

select = final_data
# select= final_data[final_data['real_root']=="Carrot Toby"]
select= select[select['real_root']=="Carrot Toby"]
# select= select[select['strain']=="C2"]
select= select[select['time_hour_binned']<=160]
select= select[select['instant_C_dens']>0]
select= select[select['instant_C_dens']<=0.03]
select= select[select['time_hour_binned']>=-40]




# sns.scatterplot(data=select,x='time_since_begin_h_shifted',y='tot_surface_area_study',size = 5,alpha = 0.3,legend=False,color="grey")

sns.lineplot(data=select,x='time_hour_binned',y='instant_BV_dens',hue = "strain",estimator = "median",palette=strain_to_color)
ax.set_ylabel('Instant biovolume density \n ($\mu m^3/mm^2$)')
ax.set_xlabel('time (hour)')
# ax.set_ylim(0,60000)
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots()

select = final_data
# select= final_data[final_data['real_root']=="Carrot Toby"]
select= select[select['real_root']=="Carrot Toby"]
# select= select[select['strain']=="C2"]
select= select[select['time_hour_binned']<=160]
select= select[select['instant_C_dens']>0]
select= select[select['instant_C_dens']<=0.03]
select= select[select['time_hour_binned']>=-40]




# sns.scatterplot(data=select,x='time_since_begin_h_shifted',y='tot_surface_area_study',size = 5,alpha = 0.3,legend=False,color="grey")

sns.lineplot(data=select,x='time_hour_binned',y='hull_speed',hue = "strain",estimator = "median",palette=strain_to_color)
ax.set_ylabel('Instant biovolume density \n ($\mu m^3/mm^2$)')
ax.set_xlabel('time (hour)')
# ax.set_ylim(0,60000)
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots()

select = final_data
# select= final_data[final_data['real_root']=="Carrot Toby"]
select= select[select['real_root']=="Carrot Toby"]
# select= select[select['strain']=="C2"]
select= select[select['time_hour_binned']<=160]
select= select[select['instant_SA_dens']<=8e4]
select= select[select['instant_SA_dens']>0]
select= select[select['time_hour_binned']>=-40]




# sns.scatterplot(data=select,x='time_since_begin_h_shifted',y='tot_surface_area_study',size = 5,alpha = 0.3,legend=False,color="grey")

sns.lineplot(data=select,x='time_hour_binned',y='hull_speed',hue = "strain",estimator = "median",palette=strain_to_color)
ax.set_ylabel('Instant biovolume density \n ($\mu m^3/mm^2$)')
ax.set_xlabel('time (hour)')
# ax.set_ylim(0,60000)
plt.tight_layout()


In [ ]:
selectd= select[select['strain']=="C3"]
len(selectd)